# Exploring low-level `PyPestUtils` library functions with a structured Freyberg model

In [ ]:
import os
import sys
import shutil
import subprocess as sp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import flopy

Setup the model dir

In [ ]:
org_d = "freyberg_monthly"
w_d = "freyberg_explore_lowlevel"
if os.path.exists(w_d):
    shutil.rmtree(w_d)
shutil.copytree(org_d,w_d)

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=w_d)
m = sim.get_model()

In [ ]:
m.dis.top.plot()

## Initial steps with the library

Install a grid from an MF6 binary grid file (ie .grb):


In [ ]:
from pypestutils.pestutilslib import PestUtilsLib
lib = PestUtilsLib()

In [ ]:
grid_info = lib.install_mf6_grid_from_file("structgrid",os.path.join(w_d,"freyberg6.dis.grb"))


`grid_info` tells us about the dimensions and type of grid:

In [ ]:
grid_info

Just to help us, lets define the standard structured grid dimension names - not required (all of this actually works just the same for unstructured grids)

In [ ]:
nrow = grid_info["ndim2"]
ncol = grid_info["ndim1"]
nlay = grid_info["ndim3"]

We can use the cell centroids for plotting stuff and also for the parameterization setup later...

In [ ]:
easting,northing,elev = lib.get_cell_centres_mf6("structgrid",grid_info["ncells"])

In [ ]:
easting = easting[:nrow*ncol]
northing = northing[:nrow*ncol]
Easting = easting.reshape((nrow,ncol))
Northing = northing.reshape((nrow,ncol))

## Post-processing

Now we will walk thru using `pypestutils` for a MOD2OBS-style model results post processing, where we extract values from the MODFLOW6 binary output file, spatially interpolate those results to the observation location, and then temporally align/interpolate the outputs to the observation times...

first we need to MODFLOW6 binary headsave file:

In [ ]:
hds_file = [os.path.join(w_d,f) for f in os.listdir(w_d) if f.endswith(".hds")][0]
hds_file

Now let's inquire what is in this file:

In [ ]:
depvar_file_info = lib.inquire_modflow_binary_file_specs(hds_file,hds_file+".out",31,1)

In [ ]:
depvar_file_info

We can use that `depvar_file_info` later in other function calls.  We can also load up the csv file that was created that summarizes the headsave file contents:

In [ ]:
df = pd.read_csv(hds_file+".out")
df.columns = [c.lower() for c in df.columns]
df

Now let's load up the actual observations - stored in a csv file:

In [ ]:
hdsdf = pd.read_csv(os.path.join("freyberg_aux_files","gwlevel_obs.csv"),parse_dates=["datetime"])
hdsdf

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(10,10))
for lay,ax in zip([1,3],axes):
    ax.set_aspect("equal")
    kdf = hdsdf.loc[hdsdf.layer==lay,:]
    assert kdf.shape[0] > 0
    ax.pcolormesh(Easting,Northing,m.dis.top.array)
    ax.scatter(kdf.x,kdf.y,marker="^",c="k",label="gw level loc")
    ax.legend(loc="upper left")
    ax.set_title("gw level locations in layer {0}".format(lay),loc="left")


Get unique site info - we will use this to calculate spatial interpolation factors from nearby model nodes to the observation locations:

In [ ]:
usitedf = hdsdf.groupby("site").first()
usitedf

Now lets calculate the observation interpolation factors

In [ ]:
fac_file = os.path.join(w_d,"obs_interp_fac.bin")
bln_file = fac_file.replace(".bin",".bln")

In [ ]:
results = lib.calc_mf6_interp_factors("structgrid",usitedf.x.values,usitedf.y.values,usitedf.layer.values,fac_file,"binary",bln_file)
results

You should see all 1s, which means there was successful interpolation factor calculation for all sites..lets make sure!

In [ ]:
assert 0 not in results

Ok not we can do the spatial interpolation from the nodes to the observation locations:

In [ ]:
head_results = lib.interp_from_mf6_depvar_file(hds_file,fac_file,"binary",depvar_file_info["ntime"],"head",1e+10,True,-1.0e+30,usitedf.shape[0])

In [ ]:
head_results["simstate"].shape

So `head_reults` would be the equivalent of MOD2SMP output.  You can stop here is you like...but there is more!

Now lets do the temporal alignment/interpolation with the actual observations.  First we need to convert from `datetime` to float time (in model time units) since the start of the simulation:

In [ ]:
start_datetime = pd.to_datetime("1-1-2018") 
hdsdf.loc[:,"time"] = hdsdf.datetime.apply(lambda x: x  - start_datetime).dt.days # we are losing fractional days..oh well...
hdsdf.time

Now lets also add an `isite` interger column to marks where difference sites start and stop:

In [ ]:
usite = hdsdf.site.unique()
usite.sort()
usite_dict = {s:c for s,c in zip(usite,np.arange(usite.shape[0],dtype=int))}
hdsdf.loc[:,"isite"] = hdsdf.site.apply(lambda x: usite_dict[x])
hdsdf.isite
hdsdf.sort_values(by=["isite","time"],inplace=True)
hdsdf

In [ ]:
ihead_results = lib.interp_to_obstime(head_results["nproctime"],head_results["simtime"],head_results["simstate"],1.e+10,"L",35.0,1.0e+30,hdsdf.isite.values,hdsdf.time.values)

In [ ]:
hdsdf.loc[:,"simulated"] = ihead_results

In [ ]:
hdsdf

Thats it!  MOD2OBS done...

## Parameterization

With an installed grid, we can get the centroids of the model nodes, which we need for parameterization stuff..

In [ ]:
easting,northing,elev = lib.get_cell_centres_mf6("structgrid",grid_info["ncells"])

In [ ]:
easting.shape,northing.shape

Thats the centroids of all nodes (across all layers).  Let's focus on 2-D stuff here...so we need to get the first `nrow * ncol` nodes:

In [ ]:
easting = easting[:nrow*ncol]
northing = northing[:nrow*ncol]

In [ ]:
easting.shape

In [ ]:
# cell area
area = np.ones_like(easting)
#active array
active = m.dis.idomain.array[0,:,:].flatten()
# property mean
mean = np.ones_like(easting)
# property variance
var = np.ones_like(easting)
# the variogram range
aa = np.ones_like(easting)*1000
# anisotropy
anis = np.ones_like(easting)*5
# bearing
bearing = (np.ones_like(easting) * 55)

First initial the random engine with seed

In [ ]:
lib.initialize_randgen(12345)

In [ ]:
# generate some reals

In [ ]:
transform = "none"
variogram_type = "exp"
power = 1.0 #unused
num_reals = 10
reals = lib.fieldgen2d_sva(easting,northing,area,active,mean,var,aa,anis,bearing,transform,variogram_type,power,num_reals)
reals.shape

In [ ]:
m.dis.top = reals[:,0]
m.dis.top.plot()

In [ ]:
lib2 = PestUtilsLib()
transform = "none"
variogram_type = "exp"
power = 1.0 #unused
num_reals = 10
lib2.initialize_randgen(54321)
reals = lib2.fieldgen2d_sva(easting,northing,area,active,mean,var,aa,anis,bearing,transform,variogram_type,power,num_reals)
print(reals)
plt.imshow(reals[:,0].reshape((nrow,ncol)))

Do something weird with bearing: make it a function of easting?

In [ ]:
bearing = np.add(np.ones((nrow,ncol)),np.atleast_2d(np.arange(ncol)))

In [ ]:
cb = plt.imshow(bearing)
plt.colorbar(cb)

In [ ]:
bearing = bearing.flatten()

In [ ]:
anis *= 2
aa *= 2

In [ ]:
reals = lib.fieldgen2d_sva(easting,northing,area,active,mean,var,aa,anis,bearing,transform,variogram_type,power,num_reals)
r = reals[:,0].reshape((nrow,ncol))
plt.imshow(r)

Now some pilot points with spatially varying variogram props.  Just sample the realization for pp values

In [ ]:
Easting = easting.reshape((nrow,ncol))
Northing = northing.reshape((nrow,ncol))
ppeasting,ppnorthing = [],[]
ppval = []
pp_space = 20
ib = m.dis.idomain.array[0,:,:]
half_pp_space = int(pp_space/2)
for i in range(half_pp_space,nrow,pp_space):
    for j in range(half_pp_space,ncol,pp_space):
        if ib[i,j] == 0:
            continue
        ppeasting.append(Easting[i,j])
        ppnorthing.append(Northing[i,j])
        ppval.append(r[i,j])
ppeasting = np.array(ppeasting)
ppnorthing = np.array(ppnorthing)
ppval = np.array(ppval)
ppeasting.shape,ppnorthing.shape,ppval.shape

In [ ]:
fig,ax = plt.subplots(1,1)
ax.set_aspect("equal")
ax.pcolormesh(Easting,Northing,r)
ax.set_title("realization")

ax.scatter(ppeasting,ppnorthing,marker=".",s=50,c='k')

Now calculate kriging factors

In [ ]:
max_pts = 50
min_pts = 1
search_dist = 1.e+10
aa_pp = aa * pp_space *10 #?
zone_pp = np.ones_like(ppeasting,dtype=int)
fac_file = os.path.join(w_d,"factors.bin")
from datetime import datetime
s = datetime.now()
ipts = lib.calc_kriging_factors_2d(ppeasting,ppnorthing,zone_pp,easting,northing,ib.flatten(),
                                   "exp","ordinary",aa_pp,anis,bearing,search_dist,max_pts,min_pts,fac_file,"binary")
"total points:",ipts," took:",(datetime.now() - s).total_seconds()

Interpolate to the grid.  I think if we are estimating changing the variogram properties, we have to call `calc_kriging_factors()` each time...

In [ ]:
result = lib.krige_using_file(fac_file,"binary",len(easting),"ordinary","none",np.array(ppval),np.zeros_like(easting),0)

In [ ]:
rr = result["targval"].reshape(nrow,ncol)
fig,axes = plt.subplots(1,2)
ax = axes[0]
ax.set_aspect("equal")
ax.set_title("pp interpolated array")
ax.pcolormesh(Easting,Northing,rr) #the interpolated array
ax = axes[1]
ax.set_aspect("equal")
ax.set_title("pp locs with sampled values")
id_mask = m.dis.idomain.array[0,:,:].copy().astype(float)
id_mask[id_mask!=0] = np.nan
ax.pcolormesh(Easting,Northing,id_mask)
ax.scatter(ppeasting,ppnorthing,marker=".",s=50,c=ppval)
